In [ ]:
new_nb("2024-05-29__STA-winlength")

In [1]:
include("lib/Nto1.jl")

using Revise … ✔ (0.2 s)
using Units, Nto1AdEx, ConnectionTests, ConnTestEval, MemDiskCache … ✔ (0.4 s)
using StatsBase … ✔


In [2]:
duration = 10minutes

600

---

In [3]:
ConnectionTests.STA_length = 1000;

In [4]:
Ns = [5, 20, 100, 400, 1600, 6500]
seeds = 1:5;

In [5]:
conntypes(inputs) = begin
    conntypes = []
    for (conntype, trains) in inputs
        append!(conntypes, repeat([conntype], length(trains)))
    end
    conntypes
end;

In [6]:
MemDiskCache.set_dir("2024-05-26__Fix_template-based_method")

"C:\\Users\\tfiers\\.julia\\MemDiskCache.jl\\2024-05-26__Fix_template-based_method"

In [ ]:
rows_STA_longwin() = begin
    rows = []
    for N in Ns
        for seed in seeds
            @show N seed
            sim = Nto1AdEx.sim(N, duration; seed);
            ((_, exc),
             (_, inh),
             (_, unc)) = inputs = get_trains_to_test(sim, Nₜ = length(sim.trains));
            all_trains = [exc; inh; unc]
            tvals = []
            @showprogress for train in all_trains
                push!(tvals, STA_test(sim.V, train.times))
            end
            push!(rows, (; N, seed, tvals, conntypes=conntypes(inputs)))
        end
    end
    rows
end

rows = @cached rows_STA_longwin()

Running rows_STA_longwin() … 
N = 5
seed = 1
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
N = 5
seed = 2
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
N = 5
seed = 3
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
N = 5
seed = 4
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
N = 5
seed = 5
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
N = 20
seed = 1
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
N = 20
seed = 2
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
N = 20
seed = 3
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
N = 20
seed = 4
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
N = 20
seed = 5
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
N = 100
seed = 1
Progress: 100%|█████████████████████████████████████████| Time: 0:00:22
N = 10

---

In [6]:
perfrows = []
for row in rows
    (; N, seed, tvals) = row
    sweep = sweep_threshold(tvals, row.conntypes)
    AUC = calc_AUROCs(sweep).AUC
    F1max = maximum(skipnan(sweep.F1))
    push!(perfrows, (; N, seed, method="STA-longwin", AUC, F1max))
end;

In [7]:
include("lib/df.jl")

using DataFrames … ✔ (0.5 s)


In [8]:
df = DataFrame(perfrows);

In [9]:
gdf = groupby(df, :N)
combine(gdf, nrow => :num_seeds, [:AUC, :F1max] .=> mean)

Row,N,num_seeds,AUC_mean,F1max_mean
,Int64,Int64,Float64,Float64
1,5,5,0.98,0.982
2,20,5,0.985,0.986
3,100,5,0.975,0.976
4,400,5,0.977,0.971
5,1600,5,0.884,0.83
6,6500,5,0.511,0.529
